In [1]:
import os

## Set directory
os.chdir('/hpc/group/pbenfeylab/CheWei/CW_data/genesys')

from genesys import *
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#import scvelo as scv
#import cellrank as cr
import seaborn as sns
from sklearn.utils import resample

/hpc/group/pbenfeylab/ch416/miniconda3/envs/genesys/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
## Conda Env pytorch-gpu on DCC
## pytorch version
print(torch.__version__)
## scanpy
print(sc.__version__)
## scvelo
#print(scv.__version__)
## cellrank
#print(cr.__version__)

1.11.0
1.9.6


## Cell Lineage Blueprint
#### White Blood Cells include HSCs, treated as precursor for definitive erythroid, B cells, T cells and Mast cells (if precursors for white bllod cells do not exist, set the transcriptomic profiles to zero)
#### Neuroectoderm and glia treated as precursor for CNS neurons,intermediate neuronal progenitors, eye and other, olfactory sensory neurons, oligodendrocytes, and ependymal_cells
#### mesoderm treated as precursor for Adipocytes and Muscule cells
#### Epithelial cells treated as precursor for Lung and airway
#### We remove transient cell population Primitive Erythroid and rarer cell type Testis and Adrenal from the training
#### For state that doesn't have cells captured, we sample cells equally from earier state and later state

## Load meta data

In [3]:
## Genes used 
df_gene = pd.read_csv('./Mouse_Embryo/df_gene.csv')

In [4]:
df_gene

,Unnamed: 0,gene_id,gene_type,gene_short_name,chr
0,ENSMUSG00000051951,ENSMUSG00000051951,protein_coding,Xkr4,chr1
1,ENSMUSG00000102343,ENSMUSG00000102343,lincRNA,Gm37381,chr1
2,ENSMUSG00000025900,ENSMUSG00000025900,protein_coding,Rp1,chr1
3,ENSMUSG00000025902,ENSMUSG00000025902,protein_coding,Sox17,chr1
4,ENSMUSG00000104328,ENSMUSG00000104328,lincRNA,Gm37323,chr1
...,...,...,...,...,...
24547,ENSMUSG00000065947,ENSMUSG00000065947,protein_coding,mt-Nd4l,chrM
24548,ENSMUSG00000064363,ENSMUSG00000064363,protein_coding,mt-Nd4,chrM
24549,ENSMUSG00000064367,ENSMUSG00000064367,protein_coding,mt-Nd5,chrM
24550,ENSMUSG00000064368,ENSMUSG00000064368,protein_coding,mt-Nd6,chrM


In [5]:
## Cells involved 
df_cell = pd.read_csv('./Mouse_Embryo/df_cell.csv')

In [6]:
df_cell

,Unnamed: 0,cell_id,day,embryo_id,experimental_batch,major_trajectory,celltype_update
0,run_4_P2-01A.ATTCAAGCATGTTACGCAAG-0-0,run_4_P2-01A.ATTCAAGCATGTTACGCAAG,E8.5,embryo_11,run_4,Neuroectoderm_and_glia,Dorsal telencephalon
1,run_4_P2-01A.CCGTCGATTTTATTCTGAG-0-0,run_4_P2-01A.CCGTCGATTTTATTCTGAG,E8.5,embryo_12,run_4,Mesoderm,Lateral plate and intermediate mesoderm
2,run_4_P2-01A.ATGGTAACTTAGCCGGTACC-0-0,run_4_P2-01A.ATGGTAACTTAGCCGGTACC,E8.5,embryo_12,run_4,Primitive_erythroid,Primitive erythroid cells
3,run_4_P2-01A.CCGACCAATTTGCGAGGCA-0-0,run_4_P2-01A.CCGACCAATTTGCGAGGCA,E8.5,embryo_8,run_4,Epithelial_cells,Pre-epidermal keratinocytes
4,run_4_P2-01A.ATGCTTCCTAATGCCGCTT-0-0,run_4_P2-01A.ATGCTTCCTAATGCCGCTT,E8.5,embryo_9,run_4,Mesoderm,Lateral plate and intermediate mesoderm
...,...,...,...,...,...,...,...
11441402,run_25_PD-12H_S384.AAATCGTTCTCAAGTCGACC-1-12-3,run_25_PD-12H_S384.AAATCGTTCTCAAGTCGACC,P0,embryo_74,run_25,Eye_and_other,Pituitary gland cells
11441403,run_25_PD-12H_S384.CTGCTGAATTCGCGGTTGG-1-12-3,run_25_PD-12H_S384.CTGCTGAATTCGCGGTTGG,P0,embryo_74,run_25,Intermediate_neuronal_progenitors,Upper-layer neurons
11441404,run_25_PD-12H_S384.TTAATGAGCTTCGGCCTTAC-1-12-3,run_25_PD-12H_S384.TTAATGAGCTTCGGCCTTAC,P0,embryo_74,run_25,CNS_neurons,Glutamatergic neurons
11441405,run_25_PD-12H_S384.ATATCCTACTTTATTCATTC-1-12-3,run_25_PD-12H_S384.ATATCCTACTTTATTCATTC,P0,embryo_74,run_25,Mesoderm,Dermatome


In [7]:
pd.Series(df_cell['day']).value_counts()

day
P0         2092528
E9.75       633746
E10.0       489516
E9.5        336999
E14.0       326063
E12.25      321031
E13.25      305678
E18.5       289835
E9.25       276424
E14.25      274509
E14.333     273184
E10.5       271152
E10.25      270204
E13.0       269386
E10.75      244190
E15.75      239821
E11.5       233843
E16.5       228892
E16.75      222748
E11.25      222125
E15.0       215781
E15.5       215284
E16.0       205343
E12.0       205001
E11.0       192305
E12.75      184737
E12.5       178598
E18.0       173401
E18.25      168317
E13.75      164240
E17.0       163901
E11.75      163114
E17.75      160003
E15.25      156648
E8.5        153597
E16.25      143323
E14.75      142384
E18.75      135155
E13.5       120811
E17.5       118818
E9.0        118049
E17.25       94470
E8.75        46253
Name: count, dtype: int64

In [8]:
pd.Series(df_cell['major_trajectory']).value_counts()

major_trajectory
Mesoderm                             3267338
CNS_neurons                          2106206
Neuroectoderm_and_glia               1733663
Definitive_erythroid                 1033409
Intermediate_neuronal_progenitors     628251
Epithelial_cells                      524960
Endothelium                           312029
Muscle_cells                          305003
Hepatocytes                           289569
White_blood_cells                     262022
Neural_crest_PNS_glia                 126743
Adipocytes                            114478
Primitive_erythroid                   108613
Neural_crest_PNS_neurons              103999
Eye_and_other                          93695
T_cells                                89429
Lung_and_airway                        76732
Intestine                              51796
B_cells                                44352
Olfactory_sensory_neurons              42826
Cardiomyocytes                         37205
Oligodendrocytes                      

## Load gene exp matrices

## adata1

In [9]:
adata1 = sc.read_h5ad("./Mouse_Embryo/adata_JAX_dataset_1.h5ad")

In [10]:
adata1

AnnData object with n_obs × n_vars = 2840130 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [11]:
sc.pp.filter_cells(adata1, min_counts=2500)
sc.pp.filter_cells(adata1, min_genes=2500)
# Calculate row sums for the number of genes per cell
row_sums = np.sum(adata1.X > 0 , axis=1).reshape(-1).tolist()[0]
# Calculate row sums for the number of counts per cell
row_sums_counts = np.sum(adata1.X, axis=1).reshape(-1).tolist()[0]

In [12]:
## Median genes per cell
np.median(row_sums)

2947.0

In [13]:
## Median UMI counts per cell
np.median(row_sums_counts)

6109.0

In [14]:
adata1

AnnData object with n_obs × n_vars = 481481 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [15]:
pd.Series(adata1.obs['day']).value_counts()

day
E8.5      112875
E10.0      98766
E9.75      96466
E9.5       37357
E9.25      36020
E9.0       30094
E10.5      21656
E10.25     21373
E10.75     17067
E8.75       9807
Name: count, dtype: int64

In [17]:
celltype = pd.merge(adata1.obs['cell_id'], df_cell, on='cell_id', how='left')
celltype.index = adata1.obs.index
adata1.obs['celltype.anno']=celltype['major_trajectory']

In [19]:
## Prepare labels
## high resolution: 11 clusters
#E8.5-9, E9.25-10, E10.25-10.5, E10.75-11.75, E12-12.75, E13-13.75, E14-14.333, E14.75-15.25, E15.5-16.5, E16.75-18, E18.25-P0
time_bin = adata1.obs['day'].copy()
time_bin = time_bin.replace('E8.5', 0)
time_bin = time_bin.replace('E8.75', 0)
time_bin = time_bin.replace('E9.0', 0)
time_bin = time_bin.replace('E9.25', 1)
time_bin = time_bin.replace('E9.5', 1)
time_bin = time_bin.replace('E9.75', 1)
time_bin = time_bin.replace('E10.0', 1)
time_bin = time_bin.replace('E10.25', 2)
time_bin = time_bin.replace('E10.5', 2)
time_bin = time_bin.replace('E10.75', 3)
time_bin = time_bin.replace('E11.0', 3)
time_bin = time_bin.replace('E11.25', 3)
time_bin = time_bin.replace('E11.5', 3)
time_bin = time_bin.replace('E11.75', 3)
time_bin = time_bin.replace('E12.0', 4)
time_bin = time_bin.replace('E12.25', 4)
time_bin = time_bin.replace('E12.5', 4)
time_bin = time_bin.replace('E12.75', 4)
time_bin = time_bin.replace('E13.0', 5)
time_bin = time_bin.replace('E13.25', 5)
time_bin = time_bin.replace('E13.5', 5)
time_bin = time_bin.replace('E13.75', 5)
time_bin = time_bin.replace('E14.0', 6)
time_bin = time_bin.replace('E14.25', 6)
time_bin = time_bin.replace('E14.333', 6)
time_bin = time_bin.replace('E14.75', 7)
time_bin = time_bin.replace('E15.0', 7)
time_bin = time_bin.replace('E15.25', 7)
time_bin = time_bin.replace('E15.5', 8)
time_bin = time_bin.replace('E15.75', 8)
time_bin = time_bin.replace('E16.0', 8)
time_bin = time_bin.replace('E16.25', 8)
time_bin = time_bin.replace('E16.5', 8)
time_bin = time_bin.replace('E16.75', 9)
time_bin = time_bin.replace('E17.0', 9)
time_bin = time_bin.replace('E17.25', 9)
time_bin = time_bin.replace('E17.5', 9)
time_bin = time_bin.replace('E17.75', 9)
time_bin = time_bin.replace('E18.0', 9)
time_bin = time_bin.replace('E18.25', 10)
time_bin = time_bin.replace('E18.5', 10)
time_bin = time_bin.replace('E18.75', 10)
time_bin = time_bin.replace('P0', 10)

#celltype = adata1.obs['celltype.anno']
#celltype = celltype.replace('Mesoderm', 0)
#celltype = celltype.replace('CNS_neurons', 1)
#celltype = celltype.replace('Neuroectoderm_and_glia', 2)
#celltype = celltype.replace('Epithelial_cells', 3)
#celltype = celltype.replace('Muscle_cells', 4)
#celltype = celltype.replace('Endothelium', 5)
#celltype = celltype.replace('Hepatocytes', 6)
#celltype = celltype.replace('Neural_crest_PNS_glia', 7)
#celltype = celltype.replace('Neural_crest_PNS_neurons', 8)
#celltype = celltype.replace('Cardiomyocytes', 9)
#celltype = celltype.replace('Megakaryocytes', 10)
#celltype = celltype.replace('Ependymal_cells', 11)

celltype = adata1.obs['celltype.anno']
celltype = celltype.replace('Mesoderm', 0)
celltype = celltype.replace('CNS_neurons', 1)
celltype = celltype.replace('Neuroectoderm_and_glia', 2)
celltype = celltype.replace('Definitive_erythroid', 3)
celltype = celltype.replace('Epithelial_cells', 4)
celltype = celltype.replace('Intermediate_neuronal_progenitors', 5)
celltype = celltype.replace('Muscle_cells', 6)
celltype = celltype.replace('Endothelium', 7)
celltype = celltype.replace('White_blood_cells', 8)
celltype = celltype.replace('Hepatocytes', 9)
celltype = celltype.replace('Neural_crest_PNS_glia', 10)
celltype = celltype.replace('Neural_crest_PNS_neurons', 11)
celltype = celltype.replace('Adipocytes', 12)
celltype = celltype.replace('Eye_and_other', 13)
celltype = celltype.replace('Lung_and_airway', 14)
celltype = celltype.replace('Intestine', 15)
celltype = celltype.replace('T_cells', 16)
celltype = celltype.replace('Cardiomyocytes', 17)
celltype = celltype.replace('Olfactory_sensory_neurons', 18)
celltype = celltype.replace('B_cells', 19)
celltype = celltype.replace('Mast_cells', 20)
celltype = celltype.replace('Oligodendrocytes', 21)
celltype = celltype.replace('Megakaryocytes', 22)
celltype = celltype.replace('Ependymal_cells', 23)
celltype = celltype.replace('Primitive_erythroid', 24)
celltype = celltype.replace('Testis_and_adrenal', 25) 
 
labels = adata1.obs['celltype.anno'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)


In [20]:
labels

run_4_P2-01A.ATTCAAGCATGTTACGCAAG-0          2_0
run_4_P2-01A.CCGTCGATTTTATTCTGAG-0           0_0
run_4_P2-01A.ATGCTTCCTAATGCCGCTT-0           0_0
run_4_P2-01A.CAGATCAGTGCTCAGCCGG-0           4_0
run_4_P2-01A.CGTCTTCCTTAGGCCATAG-0           0_0
                                            ... 
run_19_PD-12H_S384.ACATTCGTATACCTGGACGC-3    0_3
run_19_PD-12H_S384.TTCTAAGCTTTTCATAGGAC-3    0_2
run_19_PD-12H_S384.GTCGGCAATCCAACTAACC-3     0_3
run_19_PD-12H_S384.GTTCGCTGTCGTCGCAGCC-3     0_2
run_19_PD-12H_S384.AATCCAGTATGACCTTGATA-3    4_2
Length: 481481, dtype: object

In [21]:
adata1.obs['labels']=labels

In [22]:
pd.Series(adata1.obs['labels']).value_counts()

labels
0_1     121726
2_1      78458
0_0      59799
2_0      58347
4_1      28055
         ...  
13_3         2
5_2          2
18_3         2
16_0         1
18_2         1
Name: count, Length: 72, dtype: int64

In [23]:
# Set the maximum sample size for downsampling
max_sample_size = 500

# Initialize a new DataFrame to store the downsampled data
downsampled_obs = pd.DataFrame()

# Iterate over unique categories and perform downsampling
for category in adata1.obs['labels'].unique():
    # Select data for the current category
    category_data = adata1[adata1.obs['labels'] == category]

    # Perform downsampling to the maximum sample size
    downsampled_category = resample(category_data.obs, replace=False, n_samples=min(len(category_data), max_sample_size), random_state=42)

    # Append the downsampled category to the new DataFrame
    downsampled_obs = pd.concat([downsampled_obs, downsampled_category])

In [24]:
# Create a new AnnData object with the downsampled observations
adata1 = adata1[downsampled_obs.index, :]

In [25]:
## Remove class that < 20 cells
# Identify categories that meet the threshold
label_counts = adata1.obs['labels'].value_counts()
valid_labels = label_counts[label_counts >= 20].index

In [26]:
# Filter AnnData based on the valid categories
adata1 = adata1[adata1.obs['labels'].isin(valid_labels)]

In [27]:
## Remove cell types
adata1 = adata1[adata1.obs['celltype.anno'] != "Primitive_erythroid"]
adata1 = adata1[adata1.obs['celltype.anno'] != "Testis_and_adrenal"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Definitive_erythroid"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Intermediate_neuronal_progenitors"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "White_blood_cells"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Adipocytes"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Eye_and_other"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Lung_and_airway"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Intestine"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "T_cells"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Olfactory_sensory_neurons"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "B_cells"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Mast_cells"]
#adata1 = adata1[adata1.obs['celltype.anno'] != "Oligodendrocytes"]

In [28]:
adata1

View of AnnData object with n_obs × n_vars = 19280 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes', 'celltype.anno', 'labels'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [30]:
## Save the input data for GeneSys

with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data1.pkl", 'wb') as file_handle:
    pickle.dump(adata1, file_handle)

In [32]:
del(adata1)

## adata2

In [31]:
adata2 = sc.read_h5ad("./Mouse_Embryo/adata_JAX_dataset_2.h5ad")

In [33]:
adata2

AnnData object with n_obs × n_vars = 2560869 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [34]:
sc.pp.filter_cells(adata2, min_counts=2500)
sc.pp.filter_cells(adata2, min_genes=2500)
# Calculate row sums for the number of genes per cell
row_sums = np.sum(adata2.X > 0 , axis=1).reshape(-1).tolist()[0]
# Calculate row sums for the number of counts per cell
row_sums_counts = np.sum(adata2.X, axis=1).reshape(-1).tolist()[0]

In [35]:
np.median(row_sums)

2821.0

In [36]:
np.median(row_sums_counts)

6725.0

In [37]:
adata2

AnnData object with n_obs × n_vars = 244575 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [38]:
pd.Series(adata2.obs['day']).value_counts()

day
E12.25    40118
E11.25    32720
E12.0     29630
E12.5     27764
E12.75    24247
E11.5     22430
E13.25    21363
E11.0     14535
E13.75     9810
E13.0      8874
E11.75     6677
E13.5      6407
Name: count, dtype: int64

In [39]:
celltype = pd.merge(adata2.obs['cell_id'], df_cell, on='cell_id', how='left')
celltype.index = adata2.obs.index
adata2.obs['celltype.anno']=celltype['major_trajectory']

In [40]:
## Prepare labels
time_bin = adata2.obs['day'].copy()
time_bin = time_bin.replace('E8.5', 0)
time_bin = time_bin.replace('E8.75', 0)
time_bin = time_bin.replace('E9.0', 0)
time_bin = time_bin.replace('E9.25', 1)
time_bin = time_bin.replace('E9.5', 1)
time_bin = time_bin.replace('E9.75', 1)
time_bin = time_bin.replace('E10.0', 1)
time_bin = time_bin.replace('E10.25', 2)
time_bin = time_bin.replace('E10.5', 2)
time_bin = time_bin.replace('E10.75', 3)
time_bin = time_bin.replace('E11.0', 3)
time_bin = time_bin.replace('E11.25', 3)
time_bin = time_bin.replace('E11.5', 3)
time_bin = time_bin.replace('E11.75', 3)
time_bin = time_bin.replace('E12.0', 4)
time_bin = time_bin.replace('E12.25', 4)
time_bin = time_bin.replace('E12.5', 4)
time_bin = time_bin.replace('E12.75', 4)
time_bin = time_bin.replace('E13.0', 5)
time_bin = time_bin.replace('E13.25', 5)
time_bin = time_bin.replace('E13.5', 5)
time_bin = time_bin.replace('E13.75', 5)
time_bin = time_bin.replace('E14.0', 6)
time_bin = time_bin.replace('E14.25', 6)
time_bin = time_bin.replace('E14.333', 6)
time_bin = time_bin.replace('E14.75', 7)
time_bin = time_bin.replace('E15.0', 7)
time_bin = time_bin.replace('E15.25', 7)
time_bin = time_bin.replace('E15.5', 8)
time_bin = time_bin.replace('E15.75', 8)
time_bin = time_bin.replace('E16.0', 8)
time_bin = time_bin.replace('E16.25', 8)
time_bin = time_bin.replace('E16.5', 8)
time_bin = time_bin.replace('E16.75', 9)
time_bin = time_bin.replace('E17.0', 9)
time_bin = time_bin.replace('E17.25', 9)
time_bin = time_bin.replace('E17.5', 9)
time_bin = time_bin.replace('E17.75', 9)
time_bin = time_bin.replace('E18.0', 9)
time_bin = time_bin.replace('E18.25', 10)
time_bin = time_bin.replace('E18.5', 10)
time_bin = time_bin.replace('E18.75', 10)
time_bin = time_bin.replace('P0', 10)

celltype = adata2.obs['celltype.anno']
celltype = celltype.replace('Mesoderm', 0)
celltype = celltype.replace('CNS_neurons', 1)
celltype = celltype.replace('Neuroectoderm_and_glia', 2)
celltype = celltype.replace('Definitive_erythroid', 3)
celltype = celltype.replace('Epithelial_cells', 4)
celltype = celltype.replace('Intermediate_neuronal_progenitors', 5)
celltype = celltype.replace('Muscle_cells', 6)
celltype = celltype.replace('Endothelium', 7)
celltype = celltype.replace('White_blood_cells', 8)
celltype = celltype.replace('Hepatocytes', 9)
celltype = celltype.replace('Neural_crest_PNS_glia', 10)
celltype = celltype.replace('Neural_crest_PNS_neurons', 11)
celltype = celltype.replace('Adipocytes', 12)
celltype = celltype.replace('Eye_and_other', 13)
celltype = celltype.replace('Lung_and_airway', 14)
celltype = celltype.replace('Intestine', 15)
celltype = celltype.replace('T_cells', 16)
celltype = celltype.replace('Cardiomyocytes', 17)
celltype = celltype.replace('Olfactory_sensory_neurons', 18)
celltype = celltype.replace('B_cells', 19)
celltype = celltype.replace('Mast_cells', 20)
celltype = celltype.replace('Oligodendrocytes', 21)
celltype = celltype.replace('Megakaryocytes', 22)
celltype = celltype.replace('Ependymal_cells', 23)
celltype = celltype.replace('Primitive_erythroid', 24)
celltype = celltype.replace('Testis_and_adrenal', 25)                            
 
labels = adata2.obs['celltype.anno'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

adata2.obs['labels']=labels

# Set the maximum sample size for downsampling
max_sample_size = 500

# Initialize a new DataFrame to store the downsampled data
downsampled_obs = pd.DataFrame()

# Iterate over unique categories and perform downsampling
for category in adata2.obs['labels'].unique():
    # Select data for the current category
    category_data = adata2[adata2.obs['labels'] == category]

    # Perform downsampling to the maximum sample size
    downsampled_category = resample(category_data.obs, replace=False, n_samples=min(len(category_data), max_sample_size), random_state=42)

    # Append the downsampled category to the new DataFrame
    downsampled_obs = pd.concat([downsampled_obs, downsampled_category])

# Create a new AnnData object with the downsampled observations
adata2 = adata2[downsampled_obs.index, :]

## Remove class that < 20 cells
# Identify categories that meet the threshold
label_counts = adata2.obs['labels'].value_counts()
valid_labels = label_counts[label_counts >= 20].index

# Filter AnnData based on the valid categories
adata2 = adata2[adata2.obs['labels'].isin(valid_labels)]

## Remove cell types
adata2 = adata2[adata2.obs['celltype.anno'] != "Primitive_erythroid"]
adata2 = adata2[adata2.obs['celltype.anno'] != "Testis_and_adrenal"]

In [41]:
pd.Series(adata2.obs['labels']).value_counts()

labels
10_3    500
9_4     500
6_4     500
5_4     500
3_4     500
7_4     500
10_4    500
4_3     500
4_4     500
8_4     500
1_5     500
0_5     500
6_5     500
9_5     500
7_5     500
11_5    500
2_5     500
3_5     500
4_5     500
5_5     500
8_5     500
1_4     500
11_4    500
2_4     500
8_3     500
0_3     500
1_3     500
7_3     500
2_3     500
6_3     500
3_3     500
11_3    500
9_3     500
0_4     500
10_5    444
22_4    397
18_4    367
14_4    316
17_4    309
17_3    300
13_4    288
22_3    218
15_4    215
23_4    215
22_5    164
14_3    162
23_3    143
13_5    115
14_5    111
18_5     92
15_5     66
18_3     65
5_3      62
23_5     59
13_3     50
20_4     50
17_5     34
15_3     30
20_5     26
21_5     23
Name: count, dtype: int64

In [42]:
adata2

View of AnnData object with n_obs × n_vars = 21321 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes', 'celltype.anno', 'labels'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [43]:
## Save the input data for GeneSys

with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data2.pkl", 'wb') as file_handle:
    pickle.dump(adata2, file_handle)

In [44]:
del(adata2)

## adata3

In [45]:
adata3 = sc.read_h5ad("./Mouse_Embryo/adata_JAX_dataset_3.h5ad")

In [46]:
adata3

AnnData object with n_obs × n_vars = 2643980 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [47]:
sc.pp.filter_cells(adata3, min_counts=2500)
sc.pp.filter_cells(adata3, min_genes=2500)
# Calculate row sums for the number of genes per cell
row_sums = np.sum(adata3.X > 0 , axis=1).reshape(-1).tolist()[0]
# Calculate row sums for the number of counts per cell
row_sums_counts = np.sum(adata3.X, axis=1).reshape(-1).tolist()[0]

In [48]:
np.median(row_sums)

2985.0

In [49]:
np.median(row_sums_counts)

7384.0

In [50]:
adata3

AnnData object with n_obs × n_vars = 448334 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [51]:
pd.Series(adata3.obs['day']).value_counts()

day
E16.0      62465
E15.5      55055
E14.333    51673
E15.75     45596
E15.0      43340
E14.25     34298
E16.25     31856
E16.75     31692
E16.5      29148
E14.0      25583
E15.25     23031
E14.75     14597
Name: count, dtype: int64

In [52]:
celltype = pd.merge(adata3.obs['cell_id'], df_cell, on='cell_id', how='left')
celltype.index = adata3.obs.index
adata3.obs['celltype.anno']=celltype['major_trajectory']

In [53]:
## Prepare labels
time_bin = adata3.obs['day'].copy()
time_bin = time_bin.replace('E8.5', 0)
time_bin = time_bin.replace('E8.75', 0)
time_bin = time_bin.replace('E9.0', 0)
time_bin = time_bin.replace('E9.25', 1)
time_bin = time_bin.replace('E9.5', 1)
time_bin = time_bin.replace('E9.75', 1)
time_bin = time_bin.replace('E10.0', 1)
time_bin = time_bin.replace('E10.25', 2)
time_bin = time_bin.replace('E10.5', 2)
time_bin = time_bin.replace('E10.75', 3)
time_bin = time_bin.replace('E11.0', 3)
time_bin = time_bin.replace('E11.25', 3)
time_bin = time_bin.replace('E11.5', 3)
time_bin = time_bin.replace('E11.75', 3)
time_bin = time_bin.replace('E12.0', 4)
time_bin = time_bin.replace('E12.25', 4)
time_bin = time_bin.replace('E12.5', 4)
time_bin = time_bin.replace('E12.75', 4)
time_bin = time_bin.replace('E13.0', 5)
time_bin = time_bin.replace('E13.25', 5)
time_bin = time_bin.replace('E13.5', 5)
time_bin = time_bin.replace('E13.75', 5)
time_bin = time_bin.replace('E14.0', 6)
time_bin = time_bin.replace('E14.25', 6)
time_bin = time_bin.replace('E14.333', 6)
time_bin = time_bin.replace('E14.75', 7)
time_bin = time_bin.replace('E15.0', 7)
time_bin = time_bin.replace('E15.25', 7)
time_bin = time_bin.replace('E15.5', 8)
time_bin = time_bin.replace('E15.75', 8)
time_bin = time_bin.replace('E16.0', 8)
time_bin = time_bin.replace('E16.25', 8)
time_bin = time_bin.replace('E16.5', 8)
time_bin = time_bin.replace('E16.75', 9)
time_bin = time_bin.replace('E17.0', 9)
time_bin = time_bin.replace('E17.25', 9)
time_bin = time_bin.replace('E17.5', 9)
time_bin = time_bin.replace('E17.75', 9)
time_bin = time_bin.replace('E18.0', 9)
time_bin = time_bin.replace('E18.25', 10)
time_bin = time_bin.replace('E18.5', 10)
time_bin = time_bin.replace('E18.75', 10)
time_bin = time_bin.replace('P0', 10)

celltype = adata3.obs['celltype.anno']
celltype = celltype.replace('Mesoderm', 0)
celltype = celltype.replace('CNS_neurons', 1)
celltype = celltype.replace('Neuroectoderm_and_glia', 2)
celltype = celltype.replace('Definitive_erythroid', 3)
celltype = celltype.replace('Epithelial_cells', 4)
celltype = celltype.replace('Intermediate_neuronal_progenitors', 5)
celltype = celltype.replace('Muscle_cells', 6)
celltype = celltype.replace('Endothelium', 7)
celltype = celltype.replace('White_blood_cells', 8)
celltype = celltype.replace('Hepatocytes', 9)
celltype = celltype.replace('Neural_crest_PNS_glia', 10)
celltype = celltype.replace('Neural_crest_PNS_neurons', 11)
celltype = celltype.replace('Adipocytes', 12)
celltype = celltype.replace('Eye_and_other', 13)
celltype = celltype.replace('Lung_and_airway', 14)
celltype = celltype.replace('Intestine', 15)
celltype = celltype.replace('T_cells', 16)
celltype = celltype.replace('Cardiomyocytes', 17)
celltype = celltype.replace('Olfactory_sensory_neurons', 18)
celltype = celltype.replace('B_cells', 19)
celltype = celltype.replace('Mast_cells', 20)
celltype = celltype.replace('Oligodendrocytes', 21)
celltype = celltype.replace('Megakaryocytes', 22)
celltype = celltype.replace('Ependymal_cells', 23)
celltype = celltype.replace('Primitive_erythroid', 24)
celltype = celltype.replace('Testis_and_adrenal', 25)                            
 
labels = adata3.obs['celltype.anno'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

adata3.obs['labels']=labels

# Set the maximum sample size for downsampling
max_sample_size = 500

# Initialize a new DataFrame to store the downsampled data
downsampled_obs = pd.DataFrame()

# Iterate over unique categories and perform downsampling
for category in adata3.obs['labels'].unique():
    # Select data for the current category
    category_data = adata3[adata3.obs['labels'] == category]

    # Perform downsampling to the maximum sample size
    downsampled_category = resample(category_data.obs, replace=False, n_samples=min(len(category_data), max_sample_size), random_state=42)

    # Append the downsampled category to the new DataFrame
    downsampled_obs = pd.concat([downsampled_obs, downsampled_category])

# Create a new AnnData object with the downsampled observations
adata3 = adata3[downsampled_obs.index, :]

## Remove class that < 20 cells
# Identify categories that meet the threshold
label_counts = adata3.obs['labels'].value_counts()
valid_labels = label_counts[label_counts >= 20].index

# Filter AnnData based on the valid categories
adata3 = adata3[adata3.obs['labels'].isin(valid_labels)]

## Remove cell types
adata3 = adata3[adata3.obs['celltype.anno'] != "Primitive_erythroid"]
adata3 = adata3[adata3.obs['celltype.anno'] != "Testis_and_adrenal"]

In [54]:
adata3

View of AnnData object with n_obs × n_vars = 37765 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes', 'celltype.anno', 'labels'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [55]:
## Save the input data for GeneSys

with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data3.pkl", 'wb') as file_handle:
    pickle.dump(adata3, file_handle)

In [56]:
del(adata3)

## adata4

In [57]:
adata4 = sc.read_h5ad("./Mouse_Embryo/adata_JAX_dataset_4.h5ad")

In [58]:
adata4

AnnData object with n_obs × n_vars = 3396428 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [59]:
sc.pp.filter_cells(adata4, min_counts=2500)
sc.pp.filter_cells(adata4, min_genes=2500)
# Calculate row sums for the number of genes per cell
row_sums = np.sum(adata4.X > 0 , axis=1).reshape(-1).tolist()[0]
# Calculate row sums for the number of counts per cell
row_sums_counts = np.sum(adata4.X, axis=1).reshape(-1).tolist()[0]

In [60]:
np.median(row_sums)

2931.0

In [61]:
np.median(row_sums_counts)

8629.0

In [62]:
adata4

AnnData object with n_obs × n_vars = 351705 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [63]:
pd.Series(adata4.obs['day']).value_counts()

day
P0        144384
E17.0      40315
E17.25     36271
E18.0      31836
E17.5      25150
E18.5      20906
E17.75     20882
E18.25     17527
E18.75     14434
Name: count, dtype: int64

In [64]:
celltype = pd.merge(adata4.obs['cell_id'], df_cell, on='cell_id', how='left')
celltype.index = adata4.obs.index
adata4.obs['celltype.anno']=celltype['major_trajectory']

In [65]:
## Prepare labels
time_bin = adata4.obs['day'].copy()
time_bin = time_bin.replace('E8.5', 0)
time_bin = time_bin.replace('E8.75', 0)
time_bin = time_bin.replace('E9.0', 0)
time_bin = time_bin.replace('E9.25', 1)
time_bin = time_bin.replace('E9.5', 1)
time_bin = time_bin.replace('E9.75', 1)
time_bin = time_bin.replace('E10.0', 1)
time_bin = time_bin.replace('E10.25', 2)
time_bin = time_bin.replace('E10.5', 2)
time_bin = time_bin.replace('E10.75', 3)
time_bin = time_bin.replace('E11.0', 3)
time_bin = time_bin.replace('E11.25', 3)
time_bin = time_bin.replace('E11.5', 3)
time_bin = time_bin.replace('E11.75', 3)
time_bin = time_bin.replace('E12.0', 4)
time_bin = time_bin.replace('E12.25', 4)
time_bin = time_bin.replace('E12.5', 4)
time_bin = time_bin.replace('E12.75', 4)
time_bin = time_bin.replace('E13.0', 5)
time_bin = time_bin.replace('E13.25', 5)
time_bin = time_bin.replace('E13.5', 5)
time_bin = time_bin.replace('E13.75', 5)
time_bin = time_bin.replace('E14.0', 6)
time_bin = time_bin.replace('E14.25', 6)
time_bin = time_bin.replace('E14.333', 6)
time_bin = time_bin.replace('E14.75', 7)
time_bin = time_bin.replace('E15.0', 7)
time_bin = time_bin.replace('E15.25', 7)
time_bin = time_bin.replace('E15.5', 8)
time_bin = time_bin.replace('E15.75', 8)
time_bin = time_bin.replace('E16.0', 8)
time_bin = time_bin.replace('E16.25', 8)
time_bin = time_bin.replace('E16.5', 8)
time_bin = time_bin.replace('E16.75', 9)
time_bin = time_bin.replace('E17.0', 9)
time_bin = time_bin.replace('E17.25', 9)
time_bin = time_bin.replace('E17.5', 9)
time_bin = time_bin.replace('E17.75', 9)
time_bin = time_bin.replace('E18.0', 9)
time_bin = time_bin.replace('E18.25', 10)
time_bin = time_bin.replace('E18.5', 10)
time_bin = time_bin.replace('E18.75', 10)
time_bin = time_bin.replace('P0', 10)

celltype = adata4.obs['celltype.anno']
celltype = celltype.replace('Mesoderm', 0)
celltype = celltype.replace('CNS_neurons', 1)
celltype = celltype.replace('Neuroectoderm_and_glia', 2)
celltype = celltype.replace('Definitive_erythroid', 3)
celltype = celltype.replace('Epithelial_cells', 4)
celltype = celltype.replace('Intermediate_neuronal_progenitors', 5)
celltype = celltype.replace('Muscle_cells', 6)
celltype = celltype.replace('Endothelium', 7)
celltype = celltype.replace('White_blood_cells', 8)
celltype = celltype.replace('Hepatocytes', 9)
celltype = celltype.replace('Neural_crest_PNS_glia', 10)
celltype = celltype.replace('Neural_crest_PNS_neurons', 11)
celltype = celltype.replace('Adipocytes', 12)
celltype = celltype.replace('Eye_and_other', 13)
celltype = celltype.replace('Lung_and_airway', 14)
celltype = celltype.replace('Intestine', 15)
celltype = celltype.replace('T_cells', 16)
celltype = celltype.replace('Cardiomyocytes', 17)
celltype = celltype.replace('Olfactory_sensory_neurons', 18)
celltype = celltype.replace('B_cells', 19)
celltype = celltype.replace('Mast_cells', 20)
celltype = celltype.replace('Oligodendrocytes', 21)
celltype = celltype.replace('Megakaryocytes', 22)
celltype = celltype.replace('Ependymal_cells', 23)
celltype = celltype.replace('Primitive_erythroid', 24)
celltype = celltype.replace('Testis_and_adrenal', 25)                            
 
labels = adata4.obs['celltype.anno'].astype(str) + "_" + time_bin.astype(str)
labels = celltype.astype(str) + "_" + time_bin.astype(str)

adata4.obs['labels']=labels

# Set the maximum sample size for downsampling
max_sample_size = 500

# Initialize a new DataFrame to store the downsampled data
downsampled_obs = pd.DataFrame()

# Iterate over unique categories and perform downsampling
for category in adata4.obs['labels'].unique():
    # Select data for the current category
    category_data = adata4[adata4.obs['labels'] == category]

    # Perform downsampling to the maximum sample size
    downsampled_category = resample(category_data.obs, replace=False, n_samples=min(len(category_data), max_sample_size), random_state=42)

    # Append the downsampled category to the new DataFrame
    downsampled_obs = pd.concat([downsampled_obs, downsampled_category])

# Create a new AnnData object with the downsampled observations
adata4 = adata4[downsampled_obs.index, :]

## Remove class that < 20 cells
# Identify categories that meet the threshold
label_counts = adata4.obs['labels'].value_counts()
valid_labels = label_counts[label_counts >= 20].index

# Filter AnnData based on the valid categories
adata4 = adata4[adata4.obs['labels'].isin(valid_labels)]

## Remove cell types
adata4 = adata4[adata4.obs['celltype.anno'] != "Primitive_erythroid"]
adata4 = adata4[adata4.obs['celltype.anno'] != "Testis_and_adrenal"]

In [66]:
adata4

View of AnnData object with n_obs × n_vars = 21914 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes', 'celltype.anno', 'labels'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [67]:
## Save the input data for GeneSys

with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data4.pkl", 'wb') as file_handle:
    pickle.dump(adata4, file_handle)

In [68]:
del(adata4)

## Combine data

In [3]:
with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data1.pkl", 'rb') as file_handle:
    adata1 = pickle.load(file_handle)

In [4]:
with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data2.pkl", 'rb') as file_handle:
    adata2 = pickle.load(file_handle)

In [5]:
with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data3.pkl", 'rb') as file_handle:
    adata3 = pickle.load(file_handle)

In [6]:
with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data4.pkl", 'rb') as file_handle:
    adata4 = pickle.load(file_handle)

In [10]:
df_gene = pd.read_csv('./Mouse_Embryo/df_gene.csv')

In [11]:
df_gene

,Unnamed: 0,gene_id,gene_type,gene_short_name,chr
0,ENSMUSG00000051951,ENSMUSG00000051951,protein_coding,Xkr4,chr1
1,ENSMUSG00000102343,ENSMUSG00000102343,lincRNA,Gm37381,chr1
2,ENSMUSG00000025900,ENSMUSG00000025900,protein_coding,Rp1,chr1
3,ENSMUSG00000025902,ENSMUSG00000025902,protein_coding,Sox17,chr1
4,ENSMUSG00000104328,ENSMUSG00000104328,lincRNA,Gm37323,chr1
...,...,...,...,...,...
24547,ENSMUSG00000065947,ENSMUSG00000065947,protein_coding,mt-Nd4l,chrM
24548,ENSMUSG00000064363,ENSMUSG00000064363,protein_coding,mt-Nd4,chrM
24549,ENSMUSG00000064367,ENSMUSG00000064367,protein_coding,mt-Nd5,chrM
24550,ENSMUSG00000064368,ENSMUSG00000064368,protein_coding,mt-Nd6,chrM


In [8]:
adata1.var

,gene_id,gene_type,gene_short_name,chr
ENSMUSG00000051951,ENSMUSG00000051951,protein_coding,Xkr4,chr1
ENSMUSG00000102343,ENSMUSG00000102343,lincRNA,Gm37381,chr1
ENSMUSG00000025900,ENSMUSG00000025900,protein_coding,Rp1,chr1
ENSMUSG00000025902,ENSMUSG00000025902,protein_coding,Sox17,chr1
ENSMUSG00000104328,ENSMUSG00000104328,lincRNA,Gm37323,chr1
...,...,...,...,...
ENSMUSG00000065947,ENSMUSG00000065947,protein_coding,mt-Nd4l,chrM
ENSMUSG00000064363,ENSMUSG00000064363,protein_coding,mt-Nd4,chrM
ENSMUSG00000064367,ENSMUSG00000064367,protein_coding,mt-Nd5,chrM
ENSMUSG00000064368,ENSMUSG00000064368,protein_coding,mt-Nd6,chrM


In [12]:
adata2.var

,gene_id,gene_type,gene_short_name,chr
ENSMUSG00000051951,ENSMUSG00000051951,protein_coding,Xkr4,chr1
ENSMUSG00000102343,ENSMUSG00000102343,lincRNA,Gm37381,chr1
ENSMUSG00000025900,ENSMUSG00000025900,protein_coding,Rp1,chr1
ENSMUSG00000025902,ENSMUSG00000025902,protein_coding,Sox17,chr1
ENSMUSG00000104328,ENSMUSG00000104328,lincRNA,Gm37323,chr1
...,...,...,...,...
ENSMUSG00000065947,ENSMUSG00000065947,protein_coding,mt-Nd4l,chrM
ENSMUSG00000064363,ENSMUSG00000064363,protein_coding,mt-Nd4,chrM
ENSMUSG00000064367,ENSMUSG00000064367,protein_coding,mt-Nd5,chrM
ENSMUSG00000064368,ENSMUSG00000064368,protein_coding,mt-Nd6,chrM


In [13]:
adata3.var

,gene_id,gene_type,gene_short_name,chr
ENSMUSG00000051951,ENSMUSG00000051951,protein_coding,Xkr4,chr1
ENSMUSG00000102343,ENSMUSG00000102343,lincRNA,Gm37381,chr1
ENSMUSG00000025900,ENSMUSG00000025900,protein_coding,Rp1,chr1
ENSMUSG00000025902,ENSMUSG00000025902,protein_coding,Sox17,chr1
ENSMUSG00000104328,ENSMUSG00000104328,lincRNA,Gm37323,chr1
...,...,...,...,...
ENSMUSG00000065947,ENSMUSG00000065947,protein_coding,mt-Nd4l,chrM
ENSMUSG00000064363,ENSMUSG00000064363,protein_coding,mt-Nd4,chrM
ENSMUSG00000064367,ENSMUSG00000064367,protein_coding,mt-Nd5,chrM
ENSMUSG00000064368,ENSMUSG00000064368,protein_coding,mt-Nd6,chrM


In [14]:
adata4.var

,gene_id,gene_type,gene_short_name,chr
ENSMUSG00000051951,ENSMUSG00000051951,protein_coding,Xkr4,chr1
ENSMUSG00000102343,ENSMUSG00000102343,lincRNA,Gm37381,chr1
ENSMUSG00000025900,ENSMUSG00000025900,protein_coding,Rp1,chr1
ENSMUSG00000025902,ENSMUSG00000025902,protein_coding,Sox17,chr1
ENSMUSG00000104328,ENSMUSG00000104328,lincRNA,Gm37323,chr1
...,...,...,...,...
ENSMUSG00000065947,ENSMUSG00000065947,protein_coding,mt-Nd4l,chrM
ENSMUSG00000064363,ENSMUSG00000064363,protein_coding,mt-Nd4,chrM
ENSMUSG00000064367,ENSMUSG00000064367,protein_coding,mt-Nd5,chrM
ENSMUSG00000064368,ENSMUSG00000064368,protein_coding,mt-Nd6,chrM


In [13]:
adata = adata1.concatenate(adata2, adata3, adata4)

/hpc/group/pbenfeylab/ch416/miniconda3/envs/genesys/lib/python3.8/site-packages/anndata/_core/anndata.py:1763: FutureWarning: The AnnData.concatenate method is deprecated in favour of the anndata.concat function. Please use anndata.concat instead.

See the tutorial for concat at: https://anndata.readthedocs.io/en/latest/concatenation.html
  warnings.warn(


In [14]:
adata

AnnData object with n_obs × n_vars = 100280 × 24552
    obs: 'cell_id', 'keep', 'day', 'embryo_id', 'experimental_batch', 'n_counts', 'n_genes', 'celltype.anno', 'labels', 'batch'
    var: 'gene_id', 'gene_type', 'gene_short_name', 'chr'

In [15]:
del(adata1)
del(adata2)
del(adata3)
del(adata4)

In [16]:
## Processing all together
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.scale(adata, max_value=10)
##The integrated assay values were floored at zero and normalized to the range of zero to one
##flooring
adata.X[adata.X < 0]=0
mmin = np.amin(adata.X)
nor = (np.amax(adata.X)-mmin)
adata.X = (adata.X-mmin)/nor

In [19]:
adata.X

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06917345],
       [0.12367482, 0.        , 0.        , ..., 0.        , 0.        ,
        0.01952198],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.0552013 , 0.        ,
        0.        ],
       [0.06790724, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.08991871, 0.        ,
        0.16508825]], dtype=float32)

In [21]:
## Prepare training, validation and test set
X_train, X_test, y_train, y_test = train_test_split(
            adata.X, adata.obs['labels'], test_size=0.2, stratify=adata.obs['labels'], shuffle=True, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.2, stratify=y_train, shuffle=True, random_state=0)
data = {}
data['X_train'] = X_train
data['y_train'] = y_train
data['X_val'] = X_val
data['y_val'] = y_val
data['X_test'] = X_test
data['y_test'] = y_test

In [22]:
## Save the input data for GeneSys

with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data.pkl", 'wb') as file_handle:
    pickle.dump(data, file_handle)

In [3]:
with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data.pkl", 'rb') as file_handle:
    adata = pickle.load(file_handle)

## Check the input Data 

In [ ]:
with open("./genesys_mouse_embryo_11_clusters_500_cells_per_state_2500_GPC_UPC_data.pkl", 'rb') as file_handle:
    adata = pickle.load(file_handle)

In [5]:
adata['X_train'].shape

(64179, 24552)

In [6]:
adata['X_val'].shape

(16045, 24552)

In [8]:
new_nonzero = adata['X_val'][~(adata['X_val']==0)]

len(adata['X_val'][~(adata['X_val']==0)]) / adata['X_val'].shape[0]*adata['X_val'].shape[1]


69682049.54565285

In [10]:
len(new_nonzero)/(adata['X_val'].shape[0]*adata['X_val'].shape[1])

0.1155971601945124

In [7]:
adata['X_test'].shape

(20056, 24552)

In [26]:
## Percentage of data used
round((64179+16045+20056)/11441407,2)

0.01

In [27]:
pd.Series(data['y_train']).value_counts()

labels
5_9     640
11_9    640
10_3    640
11_3    640
6_9     640
       ... 
15_1     22
15_3     19
23_2     18
20_5     17
21_5     14
Name: count, Length: 218, dtype: int64

In [28]:
full_time_bins = np.arange(11)

In [29]:
for i in np.arange(24):
    print(i)
    ct = sorted(pd.Series(np.array([i.split('_', 1)[1] for i in data['y_train']], dtype='int')[np.array([i.split('_', 1)[0] for i in data['y_train']], dtype='int')==i]).value_counts().index)
    print([x for x in full_time_bins if x not in ct])

0
[]
1
[]
2
[]
3
[0, 1]
4
[]
5
[0, 1, 2]
6
[0]
7
[]
8
[0]
9
[]
10
[]
11
[]
12
[0, 1, 2, 3, 4, 5]
13
[0, 1, 2]
14
[0, 1, 2]
15
[2]
16
[0, 1, 2, 3, 4, 5]
17
[6]
18
[0, 1, 2]
19
[0, 1, 2, 3, 4, 5]
20
[0, 1, 2, 3]
21
[0, 1, 2, 3, 4]
22
[0]
23
[]


In [30]:
pd.Series(data['y_val']).value_counts()

labels
7_9     160
3_9     160
11_3    160
7_3     160
10_3    160
       ... 
15_1      5
23_2      5
17_5      5
20_5      4
21_5      4
Name: count, Length: 218, dtype: int64

In [31]:
pd.Series(data['y_test']).value_counts()

labels
11_3    200
2_3     200
10_3    200
15_9    200
3_9     200
       ... 
23_1      7
23_2      6
15_3      6
20_5      5
21_5      5
Name: count, Length: 218, dtype: int64